In [2]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re
import os
import shutil

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from time import sleep
from datetime import datetime

In [66]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_rows')

## Sacar links

In [18]:
url = f"https://www.autocasion.com/coches-ocasion"
browser = webdriver.Chrome()
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

browser.quit()

print(f"Total de coches: {total_coches}")
print(f"Total de páginas: {total_paginas}")

Total de coches: 105635
Total de páginas: 6214


In [26]:
data = []
total_anuncios = set()
total_fotos = set()
browser = webdriver.Chrome()
    
for i in tqdm.tqdm(range(1, total_paginas + 1)):
    url = f"https://www.autocasion.com/coches-ocasion?page={i}&direction=desc&sort=updated_at"
    browser.get(url)
    sleep(random.uniform(2,5))
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    coches_disponibles = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))
    
    if coches_disponibles == 0:
        break
        
    else:
    
        # Encontrar todos los anuncios en la página
        anuncios_pagina = soup.find_all("article", class_="anuncio")

        for anuncio in anuncios_pagina:
            try:
                link_anuncio = anuncio.find("button", class_="copiar")["rel"]
                if link_anuncio in total_anuncios:
                    continue
                    
                total_anuncios.add(link_anuncio)
                    
                link_foto = anuncio.find("img")["src"]
                total_fotos.add(link_foto)
                response_img = requests.get(link_foto, stream=True)
                response_img.raise_for_status()
                foto_binaria = response_img.content 

                data.append({
                    "link_anuncio": link_anuncio,
                    "foto_binaria": foto_binaria
                })
            except Exception as e:
                print(f"Error procesando un anuncio: {e}")
                continue

browser.quit()

# Mostrar resultados finales
print(f"Anuncios totales únicos: {len(total_anuncios)}")
print(f"Fotos totales únicas: {len(total_fotos)}")

  5%|▌         | 337/6214 [56:26<17:57:20, 11.00s/it]

Error procesando un anuncio: 404 Client Error: Not Found for url: https://images0.autocasion.com/unsafe/500x/ad/05/2656/596b4054942d4cb67835cf226ce827a0b165793a.jpeg


 10%|█         | 625/6214 [1:49:55<16:22:56, 10.55s/it]


Anuncios totales únicos: 7200
Fotos totales únicas: 7183


In [72]:
df_links = pd.DataFrame(data)
df_links.drop(columns=["foto_binaria"]).to_csv("../bin/anuncios_coches.csv", index=False)

In [85]:
n_divisiones = 5

# Dividir manualmente el DataFrame en partes iguales
tamaño_parte = len(df_links) // n_divisiones
dfs_divididos = [df_links.iloc[i * tamaño_parte:(i + 1) * tamaño_parte] for i in range(n_divisiones - 1)]
dfs_divididos.append(df_links.iloc[(n_divisiones - 1) * tamaño_parte:])

# Guardar cada parte como un archivo pickle separado
for i, df_parte in enumerate(dfs_divididos):
    df_parte.to_pickle(f"../bin/imagenes_coches/imagenes_parte_{i + 1}.pkl")
    print(f"Guardado imagenes_parte_{i + 1}.pkl con {len(df_parte)} registros.")

Guardado imagenes_parte_1.pkl con 1439 registros.
Guardado imagenes_parte_2.pkl con 1439 registros.
Guardado imagenes_parte_3.pkl con 1439 registros.
Guardado imagenes_parte_4.pkl con 1439 registros.
Guardado imagenes_parte_5.pkl con 1443 registros.


## Sacar coches

In [127]:
df_lista_coches = pd.read_csv('../bin/anuncios_coches.csv')

In [45]:
lista_jorge = df_lista_coches.iloc[:15147]
lista_miguel = df_lista_coches.iloc[15147:30294]
lista_carlos = df_lista_coches.iloc[30294:]

In [129]:
# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}\s*:\s*([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None

In [135]:
coches_df = dict()
browser = webdriver.Chrome()

for coche in tqdm.tqdm(df_lista_coches['link_anuncio'].iloc[:10]):
    #print(coche)
    
    try: 
        url_coche = coche
        browser.get(url_coche)
        soup = BeautifulSoup(browser.page_source)
        
    except: 
        continue
        
    
    try:
        if 'Vehículo vendido' in soup.find('div', class_= 'img-ficha').find('p').text.replace('X',''):
            continue
            
    except: 
        
        pass
    
    info_coche = {}
            
    try: 
        info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[4].text
        
    except: 
        info_coche['provincia'] = 'no disponible'
        
    info_coche['link_coche'] = coche
    info_coche['fecha_extraccion'] = datetime.now().strftime("%d/%m/%Y")
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
        
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        try: 
            detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
            for detalle in detalles_carroceria:
                detalle = detalle.text.replace('Medidas','').strip()

                if 'Largo' in detalle: 
                    info_coche['largo'] = detalle
                elif 'Ancho' in detalle: 
                    info_coche['ancho'] = detalle
                elif 'Alto' in detalle: 
                    info_coche['alto'] = detalle
                #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
                elif 'Capacidad del maletero' in detalle: 
                    info_coche['capacidad_maletero'] = detalle
                elif 'Carrocería' in detalle: 
                    info_coche['carroceria'] = detalle
                elif 'Número de plazas' in detalle: 
                    info_coche['num_plazas'] = detalle
                elif 'Número de puertas' in detalle: 
                    info_coche['num_puertas'] = detalle
                elif 'Batalla' in detalle: 
                    info_coche['batalla'] = detalle
                #supongo que si hay dos pesos el segundo será la masa max. autorizada
                elif 'Peso' in detalle: 
                    info_coche['peso'] = detalle
                    
        except:   
            info_coche['largo'] = 'no disponible'
            info_coche['ancho'] = 'no disponible'
            info_coche['alto'] = 'no disponible'
            info_coche['capacidad_maletero'] = 'no disponible'
            info_coche['carroceria'] = 'no disponible'
            info_coche['num_plazas'] = 'no disponible'
            info_coche['num_puertas'] = 'no disponible'
            info_coche['batalla'] = 'no disponible'
            info_coche['peso'] = 'no disponible'
                
           
        try:
            detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
            for detalle in detalles_consumos:
                detalle = ' '.join(detalle.text.replace('\n','').split())

                if 'Consumo medio' in detalle: 
                    info_coche['consumo_medio'] = detalle
                elif 'Consumo en carretera' in detalle: 
                    info_coche['consumo_carretera'] = detalle
                elif 'Consumo urbano' in detalle: 
                    info_coche['consumo_urbano'] = detalle
                elif 'CO2' in detalle: 
                    info_coche['CO2'] = detalle
                elif 'Depósito' in detalle: 
                    info_coche['deposito'] = detalle
                    
        except: 
            info_coche['consumo_medio'] = 'no disponible'
            info_coche['consumo_carretera'] = 'no disponible'
            info_coche['consumo_urbano'] = 'no disponible'
            info_coche['CO2'] = 'no disponible'
            info_coche['deposito'] = 'no disponible'        
        
        try:
            detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
            for detalle in detalles_motor:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Combustible' in detalle: 
                    info_coche['combustible'] = detalle
                elif 'Nº de cilindros' in detalle: 
                    info_coche['cilindros'] = detalle
                elif 'Cilindrada' in detalle: 
                    info_coche['cilindrada'] = detalle
                elif 'Sobrealimentación' in detalle: 
                    info_coche['sobrealimentacion'] = detalle
                    
        except: 
            info_coche['combustible'] = 'no disponible'
            info_coche['cilindros'] = 'no disponible'
            info_coche['cilindrada'] = 'no disponible'
            info_coche['sobrealimentacion'] = 'no disponible'

        try:
            detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
            for detalle in detalles_transmision:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Tipo de transmisión' in detalle: 
                    info_coche['tipo_cambio'] = detalle
                elif 'Tracción' in detalle: 
                    info_coche['traccion'] = detalle
                elif 'Número de marchas' in detalle: 
                    info_coche['num_marchas'] = detalle
                    
        except: 
            info_coche['tipo_cambio'] = 'no disponible'
            info_coche['traccion'] = 'no disponible'
            info_coche['num_marchas'] = 'no disponible'

        try:
            detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
            for detalle in detalles_prestaciones:
                detalle = detalle.text

                if 'Potencia (KW)' in detalle: 
                    info_coche['potencia_kw'] = detalle
                elif 'Potencia (CV)' in detalle: 
                    info_coche['potencia_cv'] = detalle
                elif 'Par' in detalle: 
                    info_coche['par'] = detalle
                elif 'Velocidad' in detalle: 
                    info_coche['velocidad_max'] = detalle
                elif 'Aceleración' in detalle: 
                    info_coche['aceleracion'] = detalle
                    
        except: 
            info_coche['potencia_kw'] = 'no disponible'
            info_coche['potencia_cv'] = 'no disponible'
            info_coche['par'] = 'no disponible'
            info_coche['velocidad_max'] = 'no disponible'
            info_coche['aceleracion'] = 'no disponible'
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(3, 5))


100%|██████████| 10/10 [00:56<00:00,  5.63s/it]


In [137]:
df = pd.DataFrame.from_dict(coches_df).T

In [139]:
df

,provincia,link_coche,fecha_extraccion,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carroceria,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion
0,Barcelona,https://www.autocasion.com/coches-segunda-mano...,21/11/2024,Detalles BMW X5 (2015),Matriculado: 01/2015,178000,Activalia Cars,Profesional,C,12 meses,25000 27000 7%,70450,4886 mm Largo,1938 mm Ancho,1762 mm Alto,Capacidad del maletero (l)650/1.870,CarroceríaTodo Terreno,Número de plazas5,Batalla (mm)\n2.933,Peso/Masa max. autorizado (kg)2.780,Número de puertas5,"Consumo medio 5,90 litros/100 km","Consumo en carretera 5,50 litros/100 km","Consumo urbano 6,70 litros/100 km",CO2 (g/km) 156 g/km,"Depósito de combustible (l) 85,00 litros",Combustible Diesel,Nº de cilindros 6 cilindros,Cilindrada (cm3) 2.993,Sobrealimentación Turbo de geometría variable,Tipo de transmisión Automática secuencial,TracciónTodo terreno tracción total permanente,Número de marchas8,Potencia (KW)190,Potencia (CV)258,Par (Nm)560,Velocidad máxima (km/h)230,"Aceleración 0-100 km/h (s)6,80"
1,Barcelona,https://www.autocasion.com/coches-segunda-mano...,21/11/2024,Detalles AUDI A5 (2019),Matriculado: 06/2019,155000,Activalia Cars,Profesional,C,12 meses,20975,47267,4733 mm Largo,1843 mm Ancho,1386 mm Alto,Capacidad del maletero (l)480/0,CarroceríaBerlina,Número de plazas5,Batalla (mm)\n2.824,Peso/Masa max. autorizado (kg)2.110,Número de puertas5,"Consumo medio 4,40 litros/100 km","Consumo en carretera 3,90 litros/100 km","Consumo urbano 4,90 litros/100 km",CO2 (g/km) 113 g/km,"Depósito de combustible (l) 54,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.968,Sobrealimentación Turbo de geometría variable,Tipo de transmisión Automática secuencial,TracciónTracción delantera,Número de marchas7,Potencia (KW)110,Potencia (CV)150,Par (Nm)320,Velocidad máxima (km/h)217,"Aceleración 0-100 km/h (s)8,90"
2,Madrid,https://www.autocasion.com/coches-segunda-mano...,21/11/2024,Detalles VOLKSWAGEN Tiguan (2020),Matriculado: 01/2020,116315,BARRAGAN Y FUENTES AUTOMOCION,Profesional,C,12 meses,29990,48938,4701 mm Largo,1839 mm Ancho,1674 mm Alto,Capacidad del maletero (l)230/1.760,CarroceríaTodo Terreno,Número de plazas7,Batalla (mm)\n2.787,Peso/Masa max. autorizado (kg)2.420,Número de puertas5,"Consumo medio 5,90 litros/100 km","Consumo en carretera 5,30 litros/100 km","Consumo urbano 6,70 litros/100 km",CO2 (g/km) 147 g/km,"Depósito de combustible (l) 60,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.968,Sobrealimentación Turbo de geometría variable,Tipo de transmisión Automática secuencial,TracciónTodo terreno tracción total permanente,Número de marchas7,Potencia (KW)140,Potencia (CV)191,Par (Nm)400,Velocidad máxima (km/h)210,"Aceleración 0-100 km/h (s)8,60"
3,Barcelona,https://www.autocasion.com/coches-segunda-mano...,21/11/2024,Detalles PORSCHE 911 (2021),Matriculado: 04/2021,49900,"WORLD CARS MARCOS, S.L.",Profesional,C,12 meses,185900,247021,4535 mm Largo,1900 mm Ancho,1303 mm Alto,Capacidad del maletero (l)128/264,CarroceríaCoupe,Número de plazas4,Batalla (mm)\n2.450,Peso/Masa max. autorizado (kg)2.040,Número de puertas2,"Consumo medio 0,00 litros/100 km","Consumo en carretera 0,00 litros/100 km","Consumo urbano 0,00 litros/100 km",CO2 (g/km) - g/km,"Depósito de combustible (l) 67,00 litros",Combustible Gasolina,Nº de cilindros 6 cilindros,Cilindrada (cm3) 3.745,Sobrealimentación Doble turbo,Tipo de transmisión Automática secuencial,TracciónTracción total permanente,Número de marchas8,Potencia (KW)427,Potencia (CV)580,Par (Nm)750,Velocidad máxima (km/h)320,"Aceleración 0-100 km/h (s)2,80"
4,Segovia,https://www.autocasion.com/coches-segunda-mano...,21/11/2024,D

In [141]:
df.to_csv('../bin/data_prueba_img.csv', index=False)

## Extracción listado marcas

In [44]:
url = 'https://www.autocasion.com/marcas'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [66]:
marcas = soup.find('ul', class_= 'marcas').find_all('span')


listado_marcas = {}
for index,detalle in enumerate (marcas, start = 1):
    detalle = detalle.text.strip()
    
    listado_marcas[index] = detalle
    

In [68]:
listado_marcas

{1: 'Abarth',
 2: 'Aiways',
 3: 'Alfa romeo',
 4: 'Alpine',
 5: 'Aston martin',
 6: 'Audi',
 7: 'Bentley',
 8: 'Bmw',
 9: 'Byd',
 10: 'Citroen',
 11: 'Cupra',
 12: 'Dacia',
 13: 'Dfsk',
 14: 'Dongfeng',
 15: 'Dr',
 16: 'Ds',
 17: 'Evo',
 18: 'Ferrari',
 19: 'Fiat',
 20: 'Ford',
 21: 'Honda',
 22: 'Hyundai',
 23: 'Ineos',
 24: 'Invicta',
 25: 'Isuzu',
 26: 'Jaecoo',
 27: 'Jaguar',
 28: 'Jeep',
 29: 'Jiayuan',
 30: 'Kgm / ssangyong',
 31: 'Kia',
 32: 'Lamborghini',
 33: 'Lancia',
 34: 'Land-rover',
 35: 'Leapmotor',
 36: 'Levc',
 37: 'Lexus',
 38: 'Livan',
 39: 'Lotus',
 40: 'Lynk & co',
 41: 'Maserati',
 42: 'Maxus',
 43: 'Mazda',
 44: 'Mercedes-benz',
 45: 'Mg',
 46: 'Mhero',
 47: 'Mini',
 48: 'Mitsubishi',
 49: 'Morgan',
 50: 'Nissan',
 51: 'Omoda',
 52: 'Opel',
 53: 'Peugeot',
 54: 'Polestar',
 55: 'Porsche',
 56: 'Renault',
 57: 'Seat',
 58: 'Seres',
 59: 'Skoda',
 60: 'Smart',
 61: 'Subaru',
 62: 'Suzuki',
 63: 'Swm',
 64: 'Tesla',
 65: 'Toyota',
 66: 'Volkswagen',
 67: 'Volvo',
 6

In [70]:
df_marcas = pd.DataFrame(list(listado_marcas.items()), columns=['id_marca', 'nombre_marca'])

In [72]:
df_marcas.to_csv('../bin/listado_marcas.csv', index=False)

## Actualizar anuncios

In [56]:
df_lista_coches = pd.read_csv('../bin/anuncios_coches.csv', header = None)
df_lista_coches.columns = ['link']

In [33]:
def actualizar_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    detener_bucle = False
    links_nuevos = []

    for pagina in tqdm.tqdm(range(1, total_paginas +1)):
        if detener_bucle:
            break
        
        pagina_url = f'{url_pagina}{pagina}'
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("div", class_="contenido-anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            link_coche = f'{url_base}{coche_url}'
            
            if link_coche in df_lista_coches.iloc[:,0].values:
                print("Link ya existe en el dataframe, deteniendo el bucle.")
                print(link_coche)
                detener_bucle = True
                break
                
            else: 
                print(f"Nuevo link encontrado: {link_coche}")
                links_nuevos.append(link_coche)
    
        
    return links_nuevos

In [49]:
links_nuevos = actualizar_coches()

  0%|          | 1/6550 [00:01<2:28:42,  1.36s/it]

Nuevo link encontrado: https://www.autocasion.com/coches-segunda-mano/seat-leon-ocasion/leon-1-5-etsi-110kw-s-2-ref14597269
Nuevo link encontrado: https://www.autocasion.com/coches-segunda-mano/ford-focus-ocasion/focus-1-5tdci-st-line-120-ref14311530
Nuevo link encontrado: https://www.autocasion.com/coches-km0/km-0/hyundai-i20-km0/i20-1-2-mpi-n-line-1-ref14388394
Nuevo link encontrado: https://www.autocasion.com/coches-segunda-mano/mercedes-benz-clase-gla-ocasion/gla-200d-urban-7g-dct-ref14388409
Nuevo link encontrado: https://www.autocasion.com/coches-segunda-mano/nissan-qashqai-ocasion/qashqai-1-5dci-n-connecta-4x2-1-ref14388426
Nuevo link encontrado: https://www.autocasion.com/coches-segunda-mano/volkswagen-t-roc-ocasion/t-roc-2-0tdi-advance-style-85kw-ref14758886
Nuevo link encontrado: https://www.autocasion.com/coches-segunda-mano/citroen-c4-ocasion/c4-1-4-vti-tonic-ref14389314
Nuevo link encontrado: https://www.autocasion.com/coches-segunda-mano/peugeot-308-ocasion/308-1-6hdi-fap

In [66]:
df_nuevos_links = pd.DataFrame(links_nuevos, columns = ['link'])

df_lista_coches['link'] = df_lista_coches['link'].str.strip()
df_nuevos_links['link'] = df_nuevos_links['link'].str.strip()

df_lista_coches_actualizado  = pd.concat([df_nuevos_links, df_lista_coches]).drop_duplicates().reset_index(drop=True)

In [60]:
len(df_lista_coches)

45437

In [64]:
len(df_nuevos_links)

13

In [68]:
len(df_lista_coches_actualizado)

8504

In [78]:
filas_duplicadas = df_lista_coches[df_lista_coches.duplicated()]
pd.set_option('display.max_colwidth', None)
print(filas_duplicadas.head(10))


                                                                                                                                link
17                    https://www.autocasion.com/coches-segunda-mano/mercedes-benz-eqe-ocasion/350-4matic-16614d00e26387-ref12732709
18            https://www.autocasion.com/coches-segunda-mano/peugeot-rifter-ocasion/rifter-1-5bluehdi-standard-access-75-ref13264477
19                                   https://www.autocasion.com/coches-km0/km-0/mercedes-benz-eqs-km0/450-166eac81c4882f-ref14348056
33                      https://www.autocasion.com/coches-segunda-mano/volkswagen-golf-ocasion/golf-1-6tdi-edition-115-1-ref13533550
49       https://www.autocasion.com/coches-segunda-mano/mercedes-benz-clase-cla-ocasion/cla-cla-200-d-dct-166e11de8e947e-ref14288060
50   https://www.autocasion.com/coches-segunda-mano/mercedes-benz-clase-gle-ocasion/gle-gle-350-de-4matic-166d935b4a5c3a-ref14241585
81                                        https://www.autocasion.com/

In [82]:
duplicados_columna = df_lista_coches['link'].duplicated().sum()
print(duplicados_columna)

36946


In [39]:
df_lista_coches[df_lista_coches.iloc[:,0] == 'https://www.autocasion.com/coches-segunda-mano/peugeot-5008-ocasion/gt-blue-hdi-130-eat8-16712e31a999ed-ref14619471']

,0
6033,https://www.autocasion.com/coches-segunda-mano...


In [100]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    lista_prueba = lista_paginas[:3]
    for pagina_url in tqdm.tqdm(lista_prueba):
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("div", class_="contenido-anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')
            
        time.sleep(random.uniform(3,7))
    
        
    return lista_paginas, anuncios_coches

In [102]:
lista_paginas, anuncios_coches = links_coches()

100%|██████████| 3/3 [00:17<00:00,  5.85s/it]


In [104]:
from collections import Counter
contador = Counter(anuncios_coches)
duplicados = {elemento: cuenta for elemento, cuenta in contador.items() if cuenta > 1}

if duplicados:
    print("Elementos duplicados y sus frecuencias:", duplicados)
else:
    print("No hay elementos duplicados en la lista.")

Elementos duplicados y sus frecuencias: {'https://www.autocasion.com/coches-segunda-mano/volkswagen-golf-ocasion/golf-2-0tdi-sport-dsg7-150-1-ref14735081': 2, 'https://www.autocasion.com/coches-segunda-mano/porsche-panamera-ocasion/panamera-gts-sport-turismo-ref14759541': 2, 'https://www.autocasion.com/coches-segunda-mano/toyota-proace-verso-ocasion/shuttle-l1-2-0d-106kw-145cv-6mt-vx-9p-1672ca70b87ef3-ref14759513': 2, 'https://www.autocasion.com/coches-segunda-mano/toyota-proace-verso-ocasion/shuttle-l1-2-0d-106kw-145cv-6mt-vx-9p-1672ca70b5dfee-ref14759512': 2, 'https://www.autocasion.com/coches-segunda-mano/toyota-proace-verso-ocasion/shuttle-l1-2-0d-106kw-145cv-6mt-vx-9p-1672ca70b34558-ref14759511': 2, 'https://www.autocasion.com/coches-segunda-mano/toyota-proace-verso-ocasion/shuttle-l1-2-0d-106kw-145cv-6mt-vx-9p-1672ca70b15257-ref14759510': 2}


## Pruebas

In [150]:
url_base = "https://www.autocasion.com"
url = 'https://www.autocasion.com/coches-ocasion?direction=desc&sort=updated_at&page=7'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [152]:
#anuncios_coches = []
anuncios = soup.find_all("div", class_="contenido-anuncio")
for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')

In [154]:
contador = Counter(anuncios_coches)
duplicados = {elemento: cuenta for elemento, cuenta in contador.items() if cuenta > 1}

if duplicados:
    print("Elementos duplicados y sus frecuencias:", duplicados)
else:
    print("No hay elementos duplicados en la lista.")

Elementos duplicados y sus frecuencias: {'https://www.autocasion.com/coches-segunda-mano/fiat-500-ocasion/500-1-0-hybrid-dolcevita-52kw-ref14658738': 5, 'https://www.autocasion.com/coches-segunda-mano/audi-a1-ocasion/a1-sportback-1-2-tfsi-ambition-ref14738922': 5, 'https://www.autocasion.com/coches-segunda-mano/volkswagen-tiguan-ocasion/tiguan-1-4-ehibrid-life-180kw-ref14658784': 5, 'https://www.autocasion.com/coches-segunda-mano/hyundai-i30-ocasion/i30-1-6crdi-go-ref14738908': 3, 'https://www.autocasion.com/coches-segunda-mano/audi-a1-ocasion/a1-sportback-1-2-tfsi-adrenalin2-ref14758197': 5, 'https://www.autocasion.com/coches-segunda-mano/volkswagen-polo-ocasion/polo-1-6tdi-cross-90-ref14758809': 5, 'https://www.autocasion.com/coches-segunda-mano/porsche-911-ocasion/911-carrera-4-coupe-tiptronic-ref14758817': 4, 'https://www.autocasion.com/coches-segunda-mano/opel-corsa-ocasion/corsa-1-4-excellence-90-1-ref14758799': 4, 'https://www.autocasion.com/coches-segunda-mano/volkswagen-golf-o

## Sacar links beautiful soup

In [33]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

403


In [9]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    for pagina_url in tqdm.tqdm(lista_paginas):
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("div", class_="contenido-anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')
            
        time.sleep(random.uniform(1,3))
    
        
    return lista_paginas, anuncios_coches

In [11]:
lista_paginas,anuncios_coches = links_coches()

100%|██████████| 6531/6531 [5:49:42<00:00,  3.21s/it]  


In [11]:
headers = {
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
            }
session = requests.Session()

In [ ]:
coches_df = dict()
browser = webdriver.Chrome()

for coche in tqdm.tqdm(lista_jorge.iloc[:, 0]):
    print(coche)
    
    try: 
        url_coche = coche
        browser.get(url_coche)
        soup = BeautifulSoup(browser.page_source)
        response = session.get(url_coche, headers=headers)
        
    except: 
        continue
        
    #Verificamos que la respuesta es exitosa
    if response.status_code != 200:
        print(f"Error al acceder a {url_coche}, Código de estado: {response.status_code}")
        continue
    
    soup = BeautifulSoup(response.content, "html.parser")
    info_coche = {}
    
    try:
        if 'Vehículo vendido' in soup.find('div', class_= 'img-ficha').find('p').text.replace('X',''):
            continue
            
    except: 
        
        pass
            
    try: 
        info_coche['provincia'] = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[4].text
        
    except: 
        info_coche['provincia'] = 'no disponible'
        
    info_coche['marca_modelo'] = soup.find('h2').text
    info_coche['fecha_matriculacion'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-calendario').find_previous().text.strip()
    info_coche['kilometraje'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-velocimetro').find_previous().text.replace('.','').replace('km','').strip()
    
    lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')
    info_coche['nombre_vendedor'] = lista_detalles_vendedor[0]
    info_coche['tipo_vendedor'] = lista_detalles_vendedor[1].strip()
        
    try:
        #puede haber o no
        info_coche['distintivo_ambiental'] = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']
        
    except:
        info_coche['distintivo_ambiental'] = 'no disponible'
    try:
        #puede haber o no
        info_coche['garantia'] = soup.find('ul', class_= 'datos-basicos-ficha').find('span', class_= 'icon icon-file').find_previous().text.replace('Garantía: ', '').strip()
        
    except:
        info_coche['garantia'] = 'sin garantía'
    
    
    detalles_precio = soup.find('ul', class_= 'tabla-precio').find_all('li')

    for detalle in detalles_precio:
        detalle = detalle.text.replace('\n','').strip()

        if 'Precio al contado' in detalle: 
            info_coche['precio_contado'] = detalle.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
        elif 'Precio de venta nuevo' in detalle: 
            info_coche['precio_nuevo'] = detalle.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]
            
    if 'precio_nuevo' not in info_coche:
        info_coche['precio_nuevo'] = 'no disponible'
    
    div_ficha_tecnica = soup.find('div', class_='bloque ficha-tecnica') 
    div_no_eurotax = soup.find('div', class_= 'bloque no-eurotax')
    
    if div_ficha_tecnica:
        
        try: 
            detalles_carroceria = div_ficha_tecnica.find('ul', class_= 'tab-spec-1 active').find_all('li')
            for detalle in detalles_carroceria:
                detalle = detalle.text.replace('Medidas','').strip()

                if 'Largo' in detalle: 
                    info_coche['largo'] = detalle
                elif 'Ancho' in detalle: 
                    info_coche['ancho'] = detalle
                elif 'Alto' in detalle: 
                    info_coche['alto'] = detalle
                #el primer número es la capacidad del maletero normal, el segundo la capacidad del maletero con los asientos abatidos
                elif 'Capacidad del maletero' in detalle: 
                    info_coche['capacidad_maletero'] = detalle
                elif 'Carrocería' in detalle: 
                    info_coche['carroceria'] = detalle
                elif 'Número de plazas' in detalle: 
                    info_coche['num_plazas'] = detalle
                elif 'Número de puertas' in detalle: 
                    info_coche['num_puertas'] = detalle
                elif 'Batalla' in detalle: 
                    info_coche['batalla'] = detalle
                #supongo que si hay dos pesos el segundo será la masa max. autorizada
                elif 'Peso' in detalle: 
                    info_coche['peso'] = detalle
                    
        except:   
            info_coche['largo'] = 'no disponible'
            info_coche['ancho'] = 'no disponible'
            info_coche['alto'] = 'no disponible'
            info_coche['capacidad_maletero'] = 'no disponible'
            info_coche['carroceria'] = 'no disponible'
            info_coche['num_plazas'] = 'no disponible'
            info_coche['num_puertas'] = 'no disponible'
            info_coche['batalla'] = 'no disponible'
            info_coche['peso'] = 'no disponible'
                
           
        try:
            detalles_consumos = div_ficha_tecnica.find('ul', class_= 'tab-spec-2').find_all('li')
            for detalle in detalles_consumos:
                detalle = ' '.join(detalle.text.replace('\n','').split())

                if 'Consumo medio' in detalle: 
                    info_coche['consumo_medio'] = detalle
                elif 'Consumo en carretera' in detalle: 
                    info_coche['consumo_carretera'] = detalle
                elif 'Consumo urbano' in detalle: 
                    info_coche['consumo_urbano'] = detalle
                elif 'CO2' in detalle: 
                    info_coche['CO2'] = detalle
                elif 'Depósito' in detalle: 
                    info_coche['deposito'] = detalle
                    
        except: 
            info_coche['consumo_medio'] = 'no disponible'
            info_coche['consumo_carretera'] = 'no disponible'
            info_coche['consumo_urbano'] = 'no disponible'
            info_coche['CO2'] = 'no disponible'
            info_coche['deposito'] = 'no disponible'        
        
        try:
            detalles_motor = div_ficha_tecnica.find('ul', class_= 'tab-spec-3').find_all('li')
            for detalle in detalles_motor:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Combustible' in detalle: 
                    info_coche['combustible'] = detalle
                elif 'Nº de cilindros' in detalle: 
                    info_coche['cilindros'] = detalle
                elif 'Cilindrada' in detalle: 
                    info_coche['cilindrada'] = detalle
                elif 'Sobrealimentación' in detalle: 
                    info_coche['sobrealimentacion'] = detalle
                    
        except: 
            info_coche['combustible'] = 'no disponible'
            info_coche['cilindros'] = 'no disponible'
            info_coche['cilindrada'] = 'no disponible'
            info_coche['sobrealimentacion'] = 'no disponible'

        try:
            detalles_transmision = div_ficha_tecnica.find('ul', class_= 'tab-spec-4').find_all('li')
            for detalle in detalles_transmision:
                detalle = detalle.text.replace('\n', ' ').strip()

                if 'Tipo de transmisión' in detalle: 
                    info_coche['tipo_cambio'] = detalle
                elif 'Tracción' in detalle: 
                    info_coche['traccion'] = detalle
                elif 'Número de marchas' in detalle: 
                    info_coche['num_marchas'] = detalle
                    
        except: 
            info_coche['tipo_cambio'] = 'no disponible'
            info_coche['traccion'] = 'no disponible'
            info_coche['num_marchas'] = 'no disponible'

        try:
            detalles_prestaciones = div_ficha_tecnica.find('ul', class_= 'tab-spec-5').find_all('li')
            for detalle in detalles_prestaciones:
                detalle = detalle.text

                if 'Potencia (KW)' in detalle: 
                    info_coche['potencia_kw'] = detalle
                elif 'Potencia (CV)' in detalle: 
                    info_coche['potencia_cv'] = detalle
                elif 'Par' in detalle: 
                    info_coche['par'] = detalle
                elif 'Velocidad' in detalle: 
                    info_coche['velocidad_max'] = detalle
                elif 'Aceleración' in detalle: 
                    info_coche['aceleracion'] = detalle
                    
        except: 
            info_coche['potencia_kw'] = 'no disponible'
            info_coche['potencia_cv'] = 'no disponible'
            info_coche['par'] = 'no disponible'
            info_coche['velocidad_max'] = 'no disponible'
            info_coche['aceleracion'] = 'no disponible'
                
    elif div_no_eurotax: 
        
        ficha_no_eurotax = div_no_eurotax.find_all('ul')
        lista_ficha_no_eurotax = []
        for elemento in range(0,len(ficha_no_eurotax)):
            detalle = div_no_eurotax.find_all('ul')[elemento].text
    
            if "Carrocería" in detalle:
                info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería")
                info_coche['num_puertas'] = buscar_detalle(detalle, "Número de puertas")
                info_coche['num_plazas'] = buscar_detalle(detalle, "Número de plazas")
                info_coche['peso'] = buscar_detalle(detalle, "Peso")

            elif "Consumo y emisiones" in detalle: 
                info_coche['consumo_medio']= buscar_detalle(detalle, "Medio \(l/100 km\)")
                info_coche['consumo_carretera']= buscar_detalle(detalle, "Carretera \(l/100 km\)")
                info_coche['consumo_urbano']= buscar_detalle(detalle, "Urbano \(l/100 km\)")

            elif "Motor" in detalle:
                info_coche['combustible'] = buscar_detalle(detalle, "Combustible")
                info_coche['potencia_cv'] = buscar_detalle(detalle, "Potencia")
                info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada")
            elif "Transmisión" in detalle:
                info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio")
                info_coche['num_marchas'] = buscar_detalle(detalle, "Marchas")
                
    coches_df[(len(coches_df))] = info_coche
    
    time.sleep(random.uniform(3, 7))
